<a href="https://colab.research.google.com/github/jeevannani1705/Project-with-ML/blob/main/Random_Forest_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install textaugment
import nltk
nltk.download('all')

     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 1.3 MB 6.3 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 65 kB 4.4 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=c4b0a30273af58e2444a494115e2a8fbeb8f7f3ab67dbc9fa37af99021364515
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping ta

True

In [6]:
import pandas as pd
import numpy as np
from textaugment import Wordnet, EDA, word2vec, Translate
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import gensim
import re
import matplotlib.pyplot as plt
import seaborn as sns 

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
with open('/content/drive/MyDrive/Colab Notebooks/PROJECT WITH ML 2022-JEEVAN/global-headline-only.csv', 'r') as f:
    df = pd.read_csv(f)

df.head()

,data_id,event_date,year,event_type,sub_event_type,region,country,location,notes,cleaned,lemmatized,stemmed
0,8378880,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Arnaba,"On 9 July 2021, regime forces shelled Arnaba t...",regime forces shelled Arnaba town in Idleb cou...,regime force shelled arnaba town idleb country...,regim forc shell arnaba town idleb countrysid ...
1,8378881,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Balyun,"On 9 July 2021, regime forces shelled Balyun t...",regime forces shelled Balyun town and its vici...,regime force shelled balyun town vicinity idle...,regim forc shell balyun town vicin idleb count...
2,8381441,09-07-2021,2021,Protests,Peaceful protest,East Asia,South Korea,Jinju,"On 9 July 2021, local veterans' organizations,...",local veterans organizations including the Kor...,local veteran organization including korean wa...,local veteran organ includ korean war veteran ...
3,8383746,09-07-2021,2021,Riots,Violent demonstration,South Asia,Pakistan,Larkana,"On 9 July 2021, prisoners held a demonstration...",prisoners held a demonstration and set wood an...,prisoner held demonstration set wood clothes f...,prison held demonstr set wood cloth fire larka...
4,8381699,09-07-2021,2021,Violence against civilians,Attack,Southeast Asia,Myanmar,Hpa Yar Gyi,"On 9 July 2021, in Hpa Yar Gyi village (Thongw...",in Hpa Yar Gyi village the village administra...,hpa yar gyi village village administrator shot...,hpa yar gyi villag villag administr shot three...


In [9]:
df.sub_event_type.value_counts()

Peaceful protest                       329399
Armed clash                            239828
Attack                                 132202
Shelling/artillery/missile attack       64743
Mob violence                            42079
Violent demonstration                   41003
Air/drone strike                        34020
Protest with intervention               31784
Remote explosive/landmine/IED           30620
Looting/property destruction            24315
Abduction/forced disappearance          14624
Arrests                                  8668
Change to group/activity                 7925
Disrupted weapons use                    7499
Government regains territory             6148
Other                                    5353
Excessive force against protesters       3098
Grenade                                  2948
Non-state actor overtakes territory      2670
Sexual violence                          2196
Agreement                                1460
Non-violent transfer of territory 

In [10]:
df.drop_duplicates(['lemmatized']).sub_event_type.value_counts()

Peaceful protest                       305383
Armed clash                             96231
Attack                                  65363
Shelling/artillery/missile attack       33279
Violent demonstration                   24574
Mob violence                            21098
Air/drone strike                        18620
Remote explosive/landmine/IED           16110
Protest with intervention               15003
Looting/property destruction            12036
Abduction/forced disappearance           7214
Change to group/activity                 7098
Arrests                                  4238
Disrupted weapons use                    3866
Other                                    3698
Government regains territory             2424
Grenade                                  1530
Excessive force against protesters       1520
Non-state actor overtakes territory      1198
Sexual violence                          1027
Agreement                                 692
Non-violent transfer of territory 

In [11]:

df.head()

,data_id,event_date,year,event_type,sub_event_type,region,country,location,notes,cleaned,lemmatized,stemmed
0,8378880,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Arnaba,"On 9 July 2021, regime forces shelled Arnaba t...",regime forces shelled Arnaba town in Idleb cou...,regime force shelled arnaba town idleb country...,regim forc shell arnaba town idleb countrysid ...
1,8378881,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Balyun,"On 9 July 2021, regime forces shelled Balyun t...",regime forces shelled Balyun town and its vici...,regime force shelled balyun town vicinity idle...,regim forc shell balyun town vicin idleb count...
2,8381441,09-07-2021,2021,Protests,Peaceful protest,East Asia,South Korea,Jinju,"On 9 July 2021, local veterans' organizations,...",local veterans organizations including the Kor...,local veteran organization including korean wa...,local veteran organ includ korean war veteran ...
3,8383746,09-07-2021,2021,Riots,Violent demonstration,South Asia,Pakistan,Larkana,"On 9 July 2021, prisoners held a demonstration...",prisoners held a demonstration and set wood an...,prisoner held demonstration set wood clothes f...,prison held demonstr set wood cloth fire larka...
4,8381699,09-07-2021,2021,Violence against civilians,Attack,Southeast Asia,Myanmar,Hpa Yar Gyi,"On 9 July 2021, in Hpa Yar Gyi village (Thongw...",in Hpa Yar Gyi village the village administra...,hpa yar gyi village village administrator shot...,hpa yar gyi villag villag administr shot three...


In [12]:
event_types_dict = {}
for num, name in enumerate(list(df['sub_event_type'].unique())):
    event_types_dict[name] = num
    # print(name, " : ", num)

event_types_dict

event_types_dict_inverse = {k:v for v,k in event_types_dict.items()}
print(event_types_dict)
print( event_types_dict_inverse)

{'Shelling/artillery/missile attack': 0, 'Peaceful protest': 1, 'Violent demonstration': 2, 'Attack': 3, 'Air/drone strike': 4, 'Armed clash': 5, 'Mob violence': 6, 'Protest with intervention': 7, 'Excessive force against protesters': 8, 'Looting/property destruction': 9, 'Other': 10, 'Remote explosive/landmine/IED': 11, 'Government regains territory': 12, 'Abduction/forced disappearance': 13, 'Change to group/activity': 14, 'Non-state actor overtakes territory': 15, 'Arrests': 16, 'Sexual violence': 17, 'Disrupted weapons use': 18, 'Non-violent transfer of territory': 19, 'Grenade': 20, 'Headquarters or base established': 21, 'Suicide bomb': 22, 'Agreement': 23, 'Chemical weapon': 24}
{0: 'Shelling/artillery/missile attack', 1: 'Peaceful protest', 2: 'Violent demonstration', 3: 'Attack', 4: 'Air/drone strike', 5: 'Armed clash', 6: 'Mob violence', 7: 'Protest with intervention', 8: 'Excessive force against protesters', 9: 'Looting/property destruction', 10: 'Other', 11: 'Remote explosi

In [13]:
def text_to_numerical_category(name):
    num = event_types_dict[name]
    return num

print(text_to_numerical_category('Peaceful protest'))
df['labels'] = df['sub_event_type'].apply(text_to_numerical_category)

1


In [14]:
df.drop(columns=['data_id', 'event_date', 'year','event_type','region','country', 'location',	'notes','cleaned', 'stemmed'	],inplace=True)

In [15]:
from sklearn.utils import shuffle

lemm = []
labels = []
THRESH_1 = 20000
THRESH_2 = 10000

for group in df.sub_event_type.unique():
    if len(df.groupby('sub_event_type').get_group(group).lemmatized.unique()) > THRESH_1:
        lemm += (list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized.unique()))[:THRESH_1])
        labels += ([group]*THRESH_1)

    elif len(df.groupby('sub_event_type').get_group(group).lemmatized.unique()) > THRESH_2:
        lemm += (list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized.unique())))
        labels += ([group]*len(list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized.unique()))))
    else:
        lemm += list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized))
        labels += ([group]*len(list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized))))


In [16]:
len(lemm)

246320

In [17]:
len(labels)

246320

In [18]:
balanced_df = pd.DataFrame({'Text' : lemm, 'Labels': labels})

In [19]:
balanced_df.head(n=10)

,Text,Labels
0,regime force shelled fleifel town idleb countr...,Shelling/artillery/missile attack
1,august naf rebel force fired grenade launcher ...,Shelling/artillery/missile attack
2,regime force shelled town bweida hama province...,Shelling/artillery/missile attack
3,regime ground force fired artillery rocket mis...,Shelling/artillery/missile attack
4,regime ground force fired rocket artillery vil...,Shelling/artillery/missile attack
5,al fath al mubeen operation room shelled regim...,Shelling/artillery/missile attack
6,turkish force shelled ypg position menagh mili...,Shelling/artillery/missile attack
7,indiscriminate shell fired presumed lna force ...,Shelling/artillery/missile attack
8,naf rebel force fired grenade launcher governm...,Shelling/artillery/missile attack
9,prohouthi force claimed thwarted attack saudia...,Shelling/artillery/missile attack


In [20]:
event_types_dict_new = {}
for num, name in enumerate(list(balanced_df.Labels.unique())):
    event_types_dict_new[name] = num
    # print(name, " : ", num)

event_types_dict_new

event_types_dict_inverse_new = {k:v for v,k in event_types_dict_new.items()}
print(event_types_dict_new)
print( event_types_dict_inverse_new)

{'Shelling/artillery/missile attack': 0, 'Peaceful protest': 1, 'Violent demonstration': 2, 'Attack': 3, 'Air/drone strike': 4, 'Armed clash': 5, 'Mob violence': 6, 'Protest with intervention': 7, 'Excessive force against protesters': 8, 'Looting/property destruction': 9, 'Other': 10, 'Remote explosive/landmine/IED': 11, 'Government regains territory': 12, 'Abduction/forced disappearance': 13, 'Change to group/activity': 14, 'Non-state actor overtakes territory': 15, 'Arrests': 16, 'Sexual violence': 17, 'Disrupted weapons use': 18, 'Non-violent transfer of territory': 19, 'Grenade': 20, 'Headquarters or base established': 21, 'Suicide bomb': 22, 'Agreement': 23, 'Chemical weapon': 24}
{0: 'Shelling/artillery/missile attack', 1: 'Peaceful protest', 2: 'Violent demonstration', 3: 'Attack', 4: 'Air/drone strike', 5: 'Armed clash', 6: 'Mob violence', 7: 'Protest with intervention', 8: 'Excessive force against protesters', 9: 'Looting/property destruction', 10: 'Other', 11: 'Remote explosi

In [21]:
balanced_df['NumLabel'] = balanced_df.Labels.apply(text_to_numerical_category)

In [22]:
# sentences = []
# for i in range(0,len(ed)):
#   sentences.append(ed.iloc(1)[0][i])

# len(sentences)

In [23]:
# nums = []
# for i in range(0,len(ed)):
#   nums.append(ed.iloc(1)[2][i])
# len(nums)

In [24]:
set(stopwords.words('english')) 

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [25]:
def word_extraction(sentence):    
  ignore = ['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 "shan't",
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 "shouldn't",
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 "wasn't",
 'we',
 'were',
 'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves']    
  words = re.sub("[^\w]", " ",  sentence).split()    
  cleaned_text = [w.lower() 
  for w in words if w not in ignore]    
  return cleaned_text

In [26]:
# def tokenize(sentences):    
#   words = [] 
#   print(len(sentences))   
#   for sentence in sentences:   
#     #print((sentence))     
#     w = word_extraction(sentence)        
#     words.extend(w)            
#     words = sorted(list(set(words))) 

#   return words

In [27]:
#words = tokenize(sentences)

In [28]:
#len(words)

In [29]:
# import numpy
# new_balanced_df = []
# for sentence in sentences:        
#   words_temp = word_extraction(sentence)        
#   bag_vector = numpy.zeros(len(words))        
#   for w in words_temp:            
#     for i,word in enumerate(words):                
#       if word == w:                     
#         bag_vector[i] += 1                            
#   new_balanced_df.append(numpy.array(bag_vector))

In [30]:
max_classes = len(balanced_df.Labels.unique())

**Model training**

Split into x test and y test


In [31]:
max_classes = len(balanced_df.Labels.unique())

In [32]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
result = shuffle(balanced_df)

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(result.Text, result.NumLabel, test_size = 0.1, random_state = 42, stratify =result.NumLabel)


In [34]:
len(X_test)

24632

In [35]:
new_corpus = []
for i in range(0,len(X_train)):
  new_corpus.append(X_train.iloc[i])
len(new_corpus)


221688

In [36]:
new_corpus2 = []
for i in range(0,len(X_test)):
  new_corpus2.append(X_test.iloc[i])
len(new_corpus2)

24632

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000)

In [38]:

X = cv.fit_transform(new_corpus).toarray()
X_train = X

In [39]:
var = cv.fit_transform(new_corpus2).toarray()
X_test = var

In [40]:
len(X_train[0])

3000

In [41]:
X_train2 = X_train
Y_train2 = Y_train

In [42]:
# X_train2 = X_train[0:100000]
# Y_train2 = Y_train[0:100000]

In [43]:
X_test2 = X_test
Y_test2 = Y_test

In [44]:
# X_test2 = X_test[0:5000]
# Y_test2 = Y_test[0:5000]

In [45]:
len(X_train2[0])

3000

**MODEL TRAINING**

Random Forest 

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [47]:
# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 300)

# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, Y_train)

# performing predictions on the test dataset
Y_pred = clf.predict(X_test)

# metrics are used to find accuracy or error
from sklearn import metrics
print()

# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(Y_test, Y_pred))


ACCURACY OF THE MODEL:  0.09053264046768432


In [48]:
print(Y_pred)
print(Y_test)

[14  1  2 ...  6  1  9]
20375      1
175536    11
173417    10
59388      2
144613     7
          ..
125076     6
194892    12
210773    13
194477    12
82286      4
Name: NumLabel, Length: 24632, dtype: int64


In [49]:
from sklearn.metrics import classification_report

print(classification_report(y_true= Y_test, y_pred= Y_pred))

              precision    recall  f1-score   support

           0       0.04      0.01      0.01      2000
           1       0.05      0.06      0.05      2000
           2       0.12      0.21      0.15      2000
           3       0.05      0.03      0.03      2000
           4       0.59      0.32      0.41      1863
           5       0.04      0.00      0.01      2000
           6       0.08      0.12      0.09      2000
           7       0.13      0.00      0.00      1501
           8       0.00      0.00      0.00       310
           9       0.06      0.15      0.09      1204
          10       0.03      0.20      0.05       535
          11       0.10      0.06      0.08      1611
          12       0.00      0.00      0.00       615
          13       0.32      0.13      0.19      1462
          14       0.04      0.14      0.06       792
          15       0.00      0.00      0.00       267
          16       0.00      0.00      0.00       867
          17       0.01    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


K FOLD 

In [1]:
!pip install matplotlib-venn

In [2]:
!apt-get -qq install -y libfluidsynth1

In [3]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

In [4]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [5]:
!pip install cartopy
import cartopy

In [10]:
import pandas as pd
import sklearn
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
clf, X_train, Y_train, cv=5, scoring='f1_macro')

NameError: ignored

In [ ]:
#results = model_selection.cross_val_score(clf, cv=kfold)

PRECISION SCORE

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = model_selection.KFold(n_splits=10, random_state=42)
model=RandomForestClassifier(n_estimators=50) 

results = model_selection.cross_val_score(estimator=model,
                                          X=X_test,
                                          y=Y_test,
                                          cv=kfold,
                                          scoring=scoring)